# Tutorial 5:  Testing generalization to new scenarios

**Week 2, Day 5: AI and Climate Change**

**By Climatematch Academy**

__Content creators:__  Deepak Mewada, Grace Lindsay
__Content reviewers:__ Name Surname, Name Surname

__Content editors:__ Name Surname, Name Surname

__Production editors:__ Name Surname, Name Surname

___

# Tutorial Objectives

*Estimated timing of tutorial: 20 minutes*

In this tutorial, we will...
* Learn about the a different type of out-of-distribution test of our model
* Evaluate the model's performance


In [ ]:
# @title Tutorial slides

# @markdown These are the slides for the videos in all tutorials today


## Uncomment the code below to test your function

#from IPython.display import IFrame
#link_id = "<YOUR_LINK_ID_HERE>"

print("If you want to download the slides: 'Link to the slides'")
      # Example: https://osf.io/download/{link_id}/

#IFrame(src=f"https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{link_id}/?direct%26mode=render", width=854, height=480)



---
# **Setup**

<details>
<summary> <font color='LightGray'>Click here for overview of this section  </font></summary>

1. **Gadget (optional)**: install and import feedback gadget - not relevant for cliamtematch
2. **Import cell (required)**: imports all libraries you use in the tutorial
3. **Hidden Figure settings cell (optional)**: sets up the plotting style (copy exactly).
4. **Hidden Plotting functions cell (optional)**: should contain all functions used to create plots throughout the tutorial (so students don't waste time looking at boilerplate matplotlib but can here if they wish to). Please use only matplotlib for plotting for consistency.
5. **Hidden Data retrieval (optional)**: set up the data retrieval needed for the tutorial.
6. **Hidden Helper functions cell (optional)**: This should contain functions that students have previously used or that are very simple. Any helper functions that are being used for the first time and are important should be placed directly above the relevant text or exercise (see Tutorial 1.1 for an example).
7. **set random seed (with/out pytorch) and set GPU/CPU (optional)**: several cells that set the seed for repeatability and set the GPU for computations  


In [ ]:
# @title Import necessary libraries:

import matplotlib.pyplot as plt    # For plotting graphs
import pandas as pd                 # For data manipulation
# # Import specific machine learning models and tools
from sklearn.model_selection import train_test_split      # For splitting dataset into train and test sets
from sklearn.ensemble import RandomForestRegressor        # For Random Forest Regression
from sklearn.tree import DecisionTreeRegressor            # For Decision Tree Regression

<details>
<summary> <font color='Red'>Click here if you are running on local machine or you encounter any error while importing   </font></summary>
Please note that if you are running this code on a local machine and encounter an error while importing a library, make sure to install the library via pip. For example, if you receive a `ModuleNotFoundError: No module named 'library name` error , please run `pip install 'library name` to install the required module.

In [ ]:
# @title Helper functions { display-mode: "form" }

# If any helper functions you want to hide for clarity (that has been seen before
# or is simple/uniformative), add here
# If helper code depends on libraries that aren't used elsewhere,
# import those libaries here, rather than in the main import cell


# Load and Prepare the Data
url_Climatebench_train_val = "https://osf.io/y2pq7/download"  # Dataset URL
training_data = pd.read_csv(url_Climatebench_train_val)  # Load the training data from the provided URL
training_data.pop('scenario')  # Drop the 'scenario' column as it's just a label and won't be passed into the model
target = training_data.pop('tas_FINAL')  # Extract the target variable 'tas_FINAL' which we aim to predict

url_spatial_test_data = "https://osf.io/7tr49/download" # location of test data
spatial_test_data = pd.read_csv(url_spatial_test_data)  # Load spatial test data from the provided URL
spatial_test_data.pop('scenario')  #we will drop the `scenario` column from the data as it is just a label, but will not be passed into the model.
spatial_test_target = spatial_test_data.pop('tas_FINAL')  # Extract the target variable 'tas_FINAL'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(training_data, target, test_size=0.2, random_state=1)

# Training the model on the training data
rf_regressor = RandomForestRegressor(random_state=42,n_estimators=80,max_depth=50)
rf_regressor.fit(X_train, y_train)

spatial_test_score = rf_regressor.score(spatial_test_data,spatial_test_target)

In [ ]:
# @title Set random seed { display-mode: "form" }

# @markdown Executing `set_seed(seed=seed)` you are setting the seed

# E.g., for DL its critical to set the random seed so that students can have a
# baseline to compare their results to expected results.
# Read more here: https://pytorch.org/docs/stable/notes/randomness.html

# Call `set_seed` function in the exercises to ensure reproducibility.
import random
import numpy as np

def set_seed(seed=None):
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  print(f'Random seed {seed} has been set.')


set_seed(seed=2024)  # change 2023 with any number you like
# Set a global seed value for reproducibility
random_state = 42

---
# **Section 1:Test generalization to held-out emissions scenario**
---
in this tutorial we will:
* Test the model’s ability on a new type of out-of-distribution data

In [ ]:
# @title Video 1: Video 1 Name
from ipywidgets import widgets
from IPython.display import display, IFrame, YouTubeVideo

out2 = widgets.Output()
with out2:
  class BiliVideo(IFrame):
    def __init__(self, id, page=1, width=400, height=300, **kwargs):
      self.id=id
      src = 'https://player.bilibili.com/player.html?bvid={0}&page={1}'.format(id, page)
      super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id="", width=854, height=480, fs=1)
  print(f'Video available at https://www.bilibili.com/video/{video.id}')
  display(video)

out1 = widgets.Output()
with out1:
  video = YouTubeVideo(id="", width=854, height=480, fs=1, rel=0)
  print(f'Video available at https://youtube.com/watch?v={video.id}')
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)

Video Summary :  
* Discuss how we previously tested generalization to unseen region  
* Stress that the real utility of these emulators is the ability to run new scenarios   
* Now we will see if the model generalizes to data from a new scenario



## Section 1.1: Load the new testing (Scenario) data
Load the new dataset and print it. As you can see, the scenario for all of these datapoints is ssp245. This scenario was not included in our initial data set. According to the scenario descriptions included in the table in Tutorial 1, ssp245 represent a "medium forcing future scenario". The lat/lon locations are the same as the initial dataset (blue box region).

In [ ]:
url_scenario_test_data = "https://osf.io/pkbwx/download"  # Dataset URL
scenario_test_data = pd.read_csv(url_scenario_test_data)  # Load scenario test data from the provided URL
scenario_test_data

Now we will prepare the data to be fed into the pre-trained model.

In [ ]:
scenario_test_data.pop('scenario')  # Remove the 'scenario' column from the dataset
scenario_test_target = scenario_test_data.pop('tas_FINAL')  # Extract the target variable 'tas_FINAL'
scenario_test_data  # Display the prepared scenario test data

## Section 1.2: Evaluate the model on this new (scenario) data

Now let's evaluate our pre-trained model(`rf_regressor`) to see how well it performs on this new emissions scenario. Use what you know to evaluate the performance and make a scatter plot of predicted vs. true temperature values.

In [ ]:
def evaluate_and_plot_scenario_performance(rf_regressor, scenario_test_data, scenario_test_target):
    """Evaluate the performance of a pre-trained model on a new emissions scenario and create a scatter plot.

    Args:
        rf_regressor (RandomForestRegressor): Pre-trained Random Forest regressor model.
        scenario_test_data (ndarray): Test features for the new emissions scenario.
        scenario_test_target (ndarray): Test labels for the new emissions scenario.

    Returns:
        float: Score of the model on the scenario test data.
    """
    #################################################
    ## TODO for students: details of what they should do ##
    # Evaluate the performance of the pre-trained model (`rf_regressor`) on the new emissions scenario
    # Calculate the score using .score() method and print it out
    # Use rf_regressor to predict temperature values for the scenario test data
    # Create a scatter plot of predicted vs. true temperature values
    # Use plt.scatter() to plot the data points
    # Use plt.plot() to plot the diagonal line y=x
    # Label the x-axis as 'Predicted Temperatures' and y-axis as 'True Temperatures'
    # Fill remove the following line of code once you have completed the exercise:
    raise NotImplementedError("Student exercise: Implement the evaluation and plotting process.")
    #################################################

    # Evaluate the performance of the pre-trained model on the new emissions scenario
    scenario_test_score = ...

    print("Scenario Test Score:", scenario_test_score)

    # Use rf_regressor to predict temperature values for the scenario test data
    scenario_test_predicted = ...

    # Create a scatter plot of predicted vs. true temperature values
    plt.scatter(..., scenario_test_target, color='b')
    plt.plot([0, 4], [0, 4], color='r')
    plt.xlabel('Predicted Temperatures')
    plt.ylabel('True Temperatures')
    plt.title('Scatter Plot of Predicted vs. True Temperature Values')
    plt.grid(True)
    plt.show()

    return scenario_test_score

# Uncomment the code below to run this cell
# scenario_test_score = evaluate_and_plot_scenario_performance(rf_regressor, scenario_test_data, scenario_test_target)


In [ ]:
# to_remove solution

def evaluate_and_plot_scenario_performance(rf_regressor, scenario_test_data, scenario_test_target):
    """Evaluate the performance of the pre-trained model on the new emissions scenario
    and create a scatter plot of predicted vs. true temperature values.

    Args:
        rf_regressor (RandomForestRegressor): Pre-trained Random Forest regressor model.
        scenario_test_data (ndarray): Test features for the new emissions scenario.
        scenario_test_target (ndarray): True temperature values for the new emissions scenario.

    Returns:
        float: Score of the model on the new emissions scenario.
    """

    # Evaluate the model on the new emissions scenario
    scenario_test_score = rf_regressor.score(scenario_test_data, scenario_test_target)
    print("Scenario Test Score:", scenario_test_score)

    # Predict temperature values for the new emissions scenario
    scenario_test_predicted = rf_regressor.predict(scenario_test_data)

    # Create a scatter plot of predicted vs. true temperature values
    plt.figure(figsize=(8, 6))
    plt.scatter(scenario_test_predicted, scenario_test_target, color='b')
    plt.plot([min(scenario_test_predicted), max(scenario_test_predicted)], [min(scenario_test_target), max(scenario_test_target)], color='r')
    plt.xlabel('Predicted Temperatures')
    plt.ylabel('True Temperatures')
    plt.title('Scatter Plot of Predicted vs. True Temperature Values for New Emissions Scenario')
    plt.grid(True)
    plt.show()

    return scenario_test_score

# Example usage:
scenario_test_score = evaluate_and_plot_scenario_performance(rf_regressor, scenario_test_data, scenario_test_target)


### Think 5.2 : Performance of the model on new Scenario data

1. Again, Have you observed the decrease in score?   
2. What do you believe could be the cause of this?   
3. What kind of new scenarios might the model perform better for?

In [ ]:
#to_remove_explanation

"""
1. Yes, there appears to be a decrease in score when the model is tested on new scenario data, though it is still well-above 0 suggesting the model has learned something about predicting temperature from emissions.
2. The decrease in score could be due to the model's inability to generalize well to new scenarios. It's possible that the model was trained on a specific set of scenarios and may not perform as accurately when presented with new, unseen scenarios. Factors such as differences in data distribution, environmental conditions, or other variables not captured in the training data could contribute to this decrease in performance.
3. The model might perform better for new scenarios that are similar to the ones it was trained on. Additionally, if the new scenarios have data distributions and patterns that are more aligned with the training data, the model could potentially perform better. However, it's important to note that the model's performance on new scenarios will ultimately depend on how well it can adapt and generalize to the differences present in the new data.
"""

For the sake of clarity let's summarize all the result.

In [ ]:
# @title  { display-mode: "form" }
# @markdown Run this cell to print the Summary of result
#
train_score = rf_regressor.score(X_train, y_train)
test_score  = rf_regressor.score(X_test, y_test)
average_score = (train_score + test_score + spatial_test_score + scenario_test_score) / 4
print(f"\tTraining Data Score                             : {train_score}")
print(f"\tTesting Data Score on same Scenario/Region      : {test_score}")
print(f"\tHeld-out Spatial Region Test Score              : {spatial_test_score}")
print(f"\tHeld-out Scenario Test Score                    : {scenario_test_score}")

This shows us that the model does generalize somewhat (i.e. the score is well above zero even in the new regions and th the new scenario). However it does not generalize very well. That is, it does not perform as well on data that differs from the data it was trained on. Ideally we would be able to build a model that inherently learns the complex relationship between emissions scenarios and future temperature. A model that truly learned this relationship would be able to generalize to new scenarios and regions.


Do you have any ideas of how to build a better machine learning model to emulate climate models? Many scientists are working on this problem!

# Summary

In this tutorial, we explored how machine learning models adapt to unfamiliar emissions scenarios. Evaluating model performance on datasets representing different emission scenarios provided insights into the models' capabilities in predicting climate variables under diverse environmental conditions.


---
# (Bonus) **Section 2: Try other Regression models**
---
Only complete this section if you are well-ahead of schedule, or have already completed the final tutorial.

Random Forest models are not the only regression models that could be applied to this problem. In this code, we will use scikit-learn to train and evaluate various regression models on the Climate Bench dataset. We will load the data, split it, define models, train them with different settings, and evaluate their performance. We will calculate and print average scores for each model configuration and identify the best-performing model.

For more information about the models used here and various other models, you can refer to [scikit-learn.org/stable/supervised_learning.html#supervised-learning](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning).  
 Note: the cell may take ~2 minutes to run.

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.linear_model import RidgeCV
import pandas as pd
from sklearn.neural_network import MLPRegressor

# Load datasets
train_val_data = pd.read_csv("https://osf.io/y2pq7/download")
spatial_test_data = pd.read_csv("https://osf.io/7tr49/download")
scenario_test_data = pd.read_csv("https://osf.io/pkbwx/download")

# Pop the 'scenario' column from all datasets
train_val_data.pop('scenario')
spatial_test_data.pop('scenario')
scenario_test_data.pop('scenario')

# Split train_val_data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_val_data.drop(columns=["tas_FINAL"]),
                                                    train_val_data["tas_FINAL"],
                                                    test_size=0.2,
                                                    random_state=1)

# Define models with different configurations
models = {
    "MLP": [make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000)),
            make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(500, 500, 500), random_state=1, max_iter=1000))],
    "RandomForest": [make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, max_depth=None)),
                     make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=50, max_depth=10))],
    "GradientBoosting": [make_pipeline(StandardScaler(), GradientBoostingRegressor(n_estimators=100, max_depth=3)),
                         make_pipeline(StandardScaler(), GradientBoostingRegressor(n_estimators=50, max_depth=2))],
    "BaggingRegressor": [make_pipeline(StandardScaler(), BaggingRegressor(n_estimators=100)),
                         make_pipeline(StandardScaler(), BaggingRegressor(n_estimators=50))],
    "SVR": [make_pipeline(StandardScaler(), SVR(kernel="linear")),
            make_pipeline(StandardScaler(), SVR(kernel="rbf"))],
    "LinearRegression": [make_pipeline(StandardScaler(), LinearRegression())],
    "Ridge": [make_pipeline(StandardScaler(), Ridge())],
    "RidgeCV":[RidgeCV(alphas=[167], cv=5)],
    "Lasso": [make_pipeline(StandardScaler(), Lasso())],
    "ElasticNet": [make_pipeline(StandardScaler(), ElasticNet())]
}

# Train models and calculate score for each configuration
results = {}
for model_name, model_list in models.items():
    model_results = []
    for config_num, model in enumerate(model_list):  # Add enumeration for configuration number
        # Train model
        model.fit(X_train, y_train)

        # Calculate scores
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)
        spatial_test_score = model.score(spatial_test_data.drop(columns=["tas_FINAL"]), spatial_test_data["tas_FINAL"])
        scenario_test_score = model.score(scenario_test_data.drop(columns=["tas_FINAL"]), scenario_test_data["tas_FINAL"])

        # Append results
        model_results.append({
            "Configuration": config_num,  # Add configuration number
            "Training Score": train_score,
            "Testing Score": test_score,
            "Spatial Test Score": spatial_test_score,
            "Scenario Test Score": scenario_test_score
        })

        # Calculate average score for the model
    average_score = sum(sum(result.values()) for result in model_results) / (len(model_results) * 4)

    # Store results including average score
    results[model_name] = {"Average Score": average_score, "Results": model_results}

# Print results including average score for each model
for model_name, model_data in results.items():
    print(f"Model: {model_name}")
    print(f"Average Score: {model_data['Average Score']}")
    print("Configuration-wise Average Scores:")
    for result in model_data['Results']:
        print(f"Configuration {result['Configuration']}: "
              f"Training Score: {result['Training Score']}, "
              f"Testing Score: {result['Testing Score']}, "
              f"Spatial Test Score: {result['Spatial Test Score']}, "
              f"Scenario Test Score: {result['Scenario Test Score']}")
    print()

# Find the best model and its average score
best_model = max(results, key=lambda x: results[x]["Average Score"])
best_average_score = results[best_model]["Average Score"]

# Print the best model and its average score
print(f"Best Model: {best_model}, Average Score: {best_average_score}")

 let's plot the result.    
Note: This code will plot the actual score for positive average scores and zero for negative average scores.

In [ ]:
# @title  { display-mode: "form" }
# @markdown Run this cell to see the plot of results!

import matplotlib.pyplot as plt

# Extract model names and average scores from results
model_names = list(results.keys())
average_scores = [results[model_name]["Average Score"] for model_name in model_names]

# Adjust scores to plot zero for negative scores
adjusted_scores = [score if score > 0 else 0 for score in average_scores]

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(model_names, adjusted_scores, color=['skyblue' if score > 0 else 'lightgray' for score in average_scores])
plt.xlabel('Model')
plt.ylabel('Average Score')
plt.title('Average Score of Different Regression Models')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()


This quick sweep of models suggests Random Forest is a good choice, but recall that most of these models have hyperparameters. Varying these hyperparameters may lead to different results!






---


                                Congratulations! You have reached the end of the  tutorial.   



---
